# Example of how to access Trends.Earth API for LandPKS

In [1]:
import json
import requests
import time
from getpass import getpass

import ipyplot
from IPython.display import Image

##  Login

In [2]:
api_url = "https://api.trends.earth"
email, password = input('Email: '), getpass('Password: ')
creds = { "email" : email, "password" : password }
auth_url = api_url + "/auth"

auth_resp = requests.post(auth_url, json=creds)
if auth_resp.status_code == 200:
    print("Logged in as {}.".format(email))
    token = auth_resp.json()['access_token']
else:
    print("Error logging in. Sever returned {}.".format(auth_resp.status_code))

Email:  azvoleff@conservation.org
Password:  ····················


Logged in as azvoleff@conservation.org.


## Submit task and poll for results

In [7]:
start_time = time.time()
params = {'user': 'azvoleff@gmail.com',
          'lang': 'ES',
          "year_start": 2001,
          "year_end": 2019,
          "geojson": "{\"coordinates\": [36.43006246, -0.71011347], \"type\": \"Point\"}",
         }
resp_run = requests.post(api_url + '/api/v1/script/landpks-1-0-4/run',
                         json=params,
                         headers={'Authorization' : 'Bearer {}'.format(token)})
print('Submission took {} seconds'.format(time.time() - start_time))

def get_results(id):
    n = 0
    while True:
        resp = requests.get(api_url + '/api/v1/execution/{}'.format(id),
                            json={'user': 'azvoleff@gmail.com'},
                            headers={'Authorization' : 'Bearer {}'.format(token)})
        if resp.json()['data']['status'] == 'FINISHED':
            return resp.json()
        elif resp.json()['data']['status'] == 'FAILED':
            return 'Failed'
        elif n > 400:
            return 'Timeout'
        time.sleep(0.2)
        n += 1
print(resp_run)
        
res = get_results(resp_run.json()['data']['id'])
print('Processing took {} seconds'.format(time.time() - start_time))
print('Return from server:')
# The required data is in the 'results' dictionary. Images will be deleted after 14 days.
#print(json.dumps(res['data']['results'], indent=4))
images = [res['data']['results']['landtrend_plot']['url']['url'],
          res['data']['results']['base_image']['url']['url'],
          res['data']['results']['greenness']['url']['url'],
          res['data']['results']['greenness_trend']['url']['url']]
ipyplot.plot_images(images, img_width=600)

Submission took 0.09800148010253906 seconds
<Response [200]>
Processing took 10.978610515594482 seconds
Return from server:


In [8]:
print(res)
print(json.dumps(res['data']['results'], indent=4))

{'data': {'end_date': '2021-07-27T21:16:59.395316', 'id': '063b5225-68b0-4fd2-8187-b7abe231a504', 'params': {'geojson': '{"coordinates": [36.43006246, -0.71011347], "type": "Point"}', 'lang': 'ES', 'user': 'azvoleff@gmail.com', 'year_end': 2019, 'year_start': 2001}, 'progress': 100, 'results': {'base_image': {'about': 'Imagen de satélite del año {YEAR_END} centrada alrededor de la ubicación del sitio', 'date': ['2019-01-01', '2019-12-31'], 'lang': 'ES', 'name': 'base_image', 'title': 'Imagen de satélite (2019)', 'type': 'ImageryPNG', 'url': {'md5Hash': 'f97c53d35f1ad6519f633dffcd50ccb9', 'url': 'https://storage.googleapis.com/ldmt/tmp3egzshcq.png'}}, 'greenness': {'about': 'Este mapa muestra el verdor anual medio para el período 2019/01/01 - 2019/12/31. Las áreas verdes más oscuras indican una vegetación más abundante y saludable durante todo el año. Los colores amarillos indican áreas con suelo desnudo o vegetación escasa.', 'date': ['2019-01-01', '2019-12-31'], 'lang': 'ES', 'name': 